<a href="https://colab.research.google.com/github/yimingm/MSSP608-Practical-Machine-Learning/blob/master/Homework2_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2 (Due February 9, 2020)
Last week you trained a machine learning classifier to predict borrower success rates on LendingClub.com, using a set of features that were already present in the dataset. This week you will spend time optimizing that model.

Feel free to reuse code from your first homework submission; this does not need to be done from scratch.

# Question 1

Choose **ONE** of the following three options:

   - The `state` column is currently broken into 51 distinct values. Create a new feature, `region`, that groups similar states together.
   - The `emp_length` column is currently a nominal value. Create a new feature, `emp_length_numeric`, that treats the variable as a numeric value instead.
   - The `title` column is a free-form string and is currently hard or impossible to use as a feature. Define one or more features that capture information about the type of loan being applied for, based on the contents of the title.

For the option you have chosen, compare how switching to your new feature in a decision tree model improves or worsens performance compared to using the original representation. Report your quantitative results along with an explanation of your new feature's possible values.

In [ ]:
import math
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import calendar
import itertools
from scipy import stats
from matplotlib import dates
from datetime import datetime

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold, GridSearchCV
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning, UndefinedMetricWarning

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


# Download the Data 
Keep this if you are working in Google Colab. Delete this if you are working on your own computer and have the data downloaded already. 

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=0B5qTk6DHjanhOV9LRE5DY3l1T2pGemVBNTVQVzVsMlFCcHF3' -O lendingclub.csv

--2020-02-09 21:58:38--  https://docs.google.com/uc?export=download&id=0B5qTk6DHjanhOV9LRE5DY3l1T2pGemVBNTVQVzVsMlFCcHF3
Resolving docs.google.com (docs.google.com)... 173.194.202.100, 173.194.202.139, 173.194.202.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.202.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nq53hl6d0pgaclurgsr2b8njfp5edv12/1581284700000/09819396713149841370/*/0B5qTk6DHjanhOV9LRE5DY3l1T2pGemVBNTVQVzVsMlFCcHF3?e=download [following]
--2020-02-09 21:58:44--  https://doc-10-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nq53hl6d0pgaclurgsr2b8njfp5edv12/1581284700000/09819396713149841370/*/0B5qTk6DHjanhOV9LRE5DY3l1T2pGemVBNTVQVzVsMlFCcHF3?e=download
Resolving doc-10-5c-docs.googleusercontent.com (doc-10-5c-docs.googleusercontent.com)... 74.125.135.132, 2607:f8b0:400e:c01::84
Connecting 

**Step 1: Open Dataset**

In [ ]:
lendingclub = pd.read_csv("lendingclub.csv") 
lendingclub.head()

,amount,date,title,fico,dti,zip,state,emp_length,policy_code,year,outcome
0,2500.0,Dec-2011,bike,740.0,1.00,309xx,GA,< 1 year,1.0,2011,accept
1,12000.0,Dec-2011,Consolidation,675.0,10.78,913xx,CA,10+ years,1.0,2011,accept
2,21000.0,Dec-2011,Debt Cleanup,705.0,13.22,335xx,FL,10+ years,1.0,2011,accept
3,31825.0,Dec-2011,Debt Consolidation Loan,760.0,14.03,080xx,NJ,5 years,1.0,2011,accept
4,12000.0,Dec-2011,Debt Consolidation,725.0,16.70,088xx,NJ,10+ years,1.0,2011,accept


**Step 2: "emp_length" Transformation**

In this step, I transformed the nominal feature indicating employment length to numeric feature. I defined "less than one year working length" as 0 working year. Since we do not know exactly how long is the "10+ years" employment length, I set the number 15 arbitrarily as the replacement of "10+ years" employment length.

New Possible Values of "emp_length":

    < 1 year = 0

    1 year = 1

    2 years = 2

    ......

    10 years = 10

    10+ years = 15


In [ ]:
def emp(x):
    if x == '< 1 year':
        return 0
    if x == '1 year':
        return 1
    if x == '2 years':
        return 2
    if x == '3 years':
        return 3
    if x == '4 years':
        return 4
    if x == '5 years':
        return 5
    if x == '6 years':
        return 6
    if x == '7 years':
        return 7
    if x == '8 years':
        return 8
    if x == '9 years':
        return 9
    if x == '10 years':
        return 10
    if x == '10+ years':
        return 15
    
lendingclub["emp"] = lendingclub["emp_length"].apply(emp)
print(lendingclub["emp"].value_counts())
lendingclub.head()

0     57065
15     5959
2      3362
1      2963
3      2690
4      2175
5      2135
6      1627
7      1243
8      1054
9       830
Name: emp, dtype: int64


,amount,date,title,fico,dti,zip,state,emp_length,policy_code,year,outcome,emp
0,2500.0,Dec-2011,bike,740.0,1.00,309xx,GA,< 1 year,1.0,2011,accept,0
1,12000.0,Dec-2011,Consolidation,675.0,10.78,913xx,CA,10+ years,1.0,2011,accept,15
2,21000.0,Dec-2011,Debt Cleanup,705.0,13.22,335xx,FL,10+ years,1.0,2011,accept,15
3,31825.0,Dec-2011,Debt Consolidation Loan,760.0,14.03,080xx,NJ,5 years,1.0,2011,accept,5
4,12000.0,Dec-2011,Debt Consolidation,725.0,16.70,088xx,NJ,10+ years,1.0,2011,accept,15


**Step 3: Build Decision Tree**

The following codes trained the decision trees and provided the performance (accuracy, kappa and precision) of the trained classifier using scikit-learn. 
Both decision trees contain features "amount" and "fico", with numeric employment length in one tree and nominal employment length in another.

In [ ]:
# Pack the training sets with 3-feature-set 
# Replace nominal employment length with numeric employment length in the second tree.
employment_numeric = ["emp"]
employment_nominal = ["emp_length"]
amount_features = ["amount"]
fico_features = ["fico"]

feature_sets = {
    "employment_nominal + amount + fico": employment_nominal + amount_features + fico_features,
    "employment_numeric + amount + fico": employment_numeric + amount_features + fico_features
}

In [ ]:
# Compare feature sets after training each feature set
best = 0
best_name = None
best_actual = None
best_predictions = None

precisions = []
kappas = []
accuracies = []

predictions = {}
actual = None

# For each feature set, we evaluate our model on both the train and the test set
for set_name, feature_set in feature_sets.items():

    # Create a dummyset with only the features in our feature set
    X = lendingclub.loc[:, feature_set]
    X = pd.get_dummies(X)

    y = lendingclub["outcome"]

    # Use scikit-learn to create our train/test split and train our decision tree
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=123)
    model = DecisionTreeClassifier(criterion="entropy", random_state=123).fit(X_train, y_train)
    
    # Calculate our accuracy, kappa and precision on the train and test sets
    y_pred = model.predict(X_test)

    accuracy = 100*accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label= "accept")

    metric_to_optimize = accuracy
    
    if metric_to_optimize > best:
        best = metric_to_optimize
        best_name = set_name
        
    predictions[set_name] = y_pred
    actual = np.array(list(y_test))
    
    # The printed result includes performance of the feature set and the confusion matrix
    print(f"Results for {set_name}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Accuracy: {accuracy:.3f} Kappa: {kappa:.3f} Precision: {precision:.3f}")
    precisions.append(precision)
    kappas.append(kappa)
    accuracies.append(accuracy)
    print("------------------------")
    
print(f"Best feature set is: {best_name} \nWith: {best:.1f}% accuracy.")    

Results for employment_nominal + amount + fico:
[[ 1493   379]
 [  424 13925]]
Accuracy: 95.050 Kappa: 0.760 Precision: 0.779
------------------------
Results for employment_numeric + amount + fico:
[[ 1497   375]
 [  428 13921]]
Accuracy: 95.050 Kappa: 0.760 Precision: 0.778
------------------------
Best feature set is: employment_nominal + amount + fico 
With: 95.0% accuracy.


**Step 4: Decision Trees Comparison**

According to the confusion matrixes and trained performances of the two model, basically nothing changed. We can say that changing employment length from nomial to numeric has no effect on the performance of the decision tree.

# Question 2

Choose **THREE** of the following five options:
1. Amount of data available for training.
2. Subset of features included in the model.
3. Decision tree vs. logistic regression classification.
4. Hyperparameter tuning for the best-performing model from option 3.
5. Number of folds or stratification strategy for cross-validation.

For each option you choose, write code to see how much impact that option has on model performance. For instance, you might choose to measure prediction accuracy as the amount of available data is increased from 1% of the total training set to 100%.

Rank the three options you chose from the greatest impact on model performance to the least impact. Justify your ranking, based on specific quantitative experimental evidence.

The following three options are chosen:
1. Amount of data available for training.
2. Subset of features included in the model.
3. Decision tree vs. logistic regression classification.


**Optimization 1: Amount of data available for training**

In order to change the amount of training data, I adjusted the proportion of test data, from 20% to 80%.

In [ ]:
# Using the same training sets from Question 1
employment_numeric = ["emp"]
employment_nominal = ["emp_length"]
amount_features = ["amount"]
fico_features = ["fico"]

feature_sets = {
    "employment_nominal + amount + fico": employment_nominal + amount_features + fico_features,
    "employment_numeric + amount + fico": employment_numeric + amount_features + fico_features
}

In [ ]:
# Compare feature sets after training each feature set
best = 0
best_name = None
best_actual = None
best_predictions = None

precisions = []
kappas = []
accuracies = []

predictions = {}
actual = None

# For each feature set, we evaluate our model on both the train and the test set
for set_name, feature_set in feature_sets.items():

    # Create a dummyset with only the features in our feature set
    X = lendingclub.loc[:, feature_set]
    X = pd.get_dummies(X)

    y = lendingclub["outcome"]

    # Use scikit-learn to create our train/test split and train our decision tree
    # Change test_size from 0.20 to 0.80
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.80, random_state=123)
    model = DecisionTreeClassifier(criterion="entropy", random_state=123).fit(X_train, y_train)
    
    # Calculate our accuracy, kappa and precision on the train and test sets
    y_pred = model.predict(X_test)

    accuracy = 100*accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label= "accept")

    metric_to_optimize = accuracy
    
    if metric_to_optimize > best:
        best = metric_to_optimize
        best_name = set_name
        
    predictions[set_name] = y_pred
    actual = np.array(list(y_test))
    
    # The printed result includes performance of the feature set and the confusion matrix
    print(f"Results for {set_name}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Accuracy: {accuracy:.3f} Kappa: {kappa:.3f} Precision: {precision:.3f}")
    precisions.append(precision)
    kappas.append(kappa)
    accuracies.append(accuracy)
    print("------------------------")
    
print(f"Best feature set is: {best_name} \nWith: {best:.1f}% accuracy.")   

Results for employment_nominal + amount + fico:
[[ 5640  1724]
 [ 2079 55440]]
Accuracy: 94.139 Kappa: 0.715 Precision: 0.731
------------------------
Results for employment_numeric + amount + fico:
[[ 5674  1690]
 [ 2112 55407]]
Accuracy: 94.140 Kappa: 0.716 Precision: 0.729
------------------------
Best feature set is: employment_numeric + amount + fico 
With: 94.1% accuracy.


Model Comparison:

With the amount of training set declined from 80% of total data to 20% of total data, the accuracy of the model declined from 95% to 94.14%, Kappa declined from 0.76 to 0.72, precison declined from 0.78 to 0.73. 

With less data available in the training set, the model performed worse.

**Optimization 2: Subset of features included in the model**

In this step, the features are mix-matched in the feature sets.

In [ ]:
employment_numeric = ["emp"]
employment_nominal = ["emp_length"]
amount_features = ["amount"]
fico_features = ["fico"]

feature_sets = {
    "employment_numeric + amount + fico": employment_numeric + amount_features + fico_features,
    "employment_numeric + amount": employment_nominal + amount_features,
    "employment_numeric + fico": employment_numeric + fico_features,
    "amount + fico": amount_features + fico_features,
    "employment_numeric": employment_numeric,
    "amount": amount_features,
    "fico": fico_features
}

In [ ]:
# Compare feature sets after training each feature set
best = 0
best_name = None
best_actual = None
best_predictions = None

precisions = []
kappas = []
accuracies = []

predictions = {}
actual = None

# For each feature set, we evaluate our model on both the train and the test set
for set_name, feature_set in feature_sets.items():

    # Create a dummyset with only the features in our feature set
    X = lendingclub.loc[:, feature_set]
    X = pd.get_dummies(X)

    y = lendingclub["outcome"]

    # Use scikit-learn to create our train/test split and train our decision tree
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=123)
    model = DecisionTreeClassifier(criterion="entropy", random_state=123).fit(X_train, y_train)
    
    # List of hyperparameter settings
    # print(model)
    # Calculate our accuracy, kappa and precision on the train and test sets
    y_pred = model.predict(X_test)

    accuracy = 100*accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label= "accept")

    metric_to_optimize = accuracy
    
    if metric_to_optimize > best:
        best = metric_to_optimize
        best_name = set_name
        
    predictions[set_name] = y_pred
    actual = np.array(list(y_test))
    
    # The printed result includes performance of the feature set and the confusion matrix
    print(f"Results for {set_name}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Accuracy: {accuracy:.1f} Kappa: {kappa:.3f} Precision: {precision:.3f}")
    precisions.append(precision)
    kappas.append(kappa)
    accuracies.append(accuracy)
    print("------------------------")
    
print(f"Best feature set is: {best_name} \nWith: {best:.1f}% accuracy.") 

Results for employment_numeric + amount + fico:
[[ 1497   375]
 [  428 13921]]
Accuracy: 95.0 Kappa: 0.760 Precision: 0.778
------------------------
Results for employment_numeric + amount:
[[  738  1134]
 [  443 13906]]
Accuracy: 90.3 Kappa: 0.433 Precision: 0.625
------------------------
Results for employment_numeric + fico:
[[ 1674   198]
 [  376 13973]]
Accuracy: 96.5 Kappa: 0.834 Precision: 0.817
------------------------
Results for amount + fico:
[[ 1377   495]
 [  491 13858]]
Accuracy: 93.9 Kappa: 0.702 Precision: 0.737
------------------------


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Results for employment_numeric:
[[    0  1872]
 [    0 14349]]
Accuracy: 88.5 Kappa: 0.000 Precision: 0.000
------------------------
Results for amount:
[[  417  1455]
 [  223 14126]]
Accuracy: 89.7 Kappa: 0.290 Precision: 0.652
------------------------
Results for fico:
[[ 1868     4]
 [ 1164 13185]]
Accuracy: 92.8 Kappa: 0.722 Precision: 0.616
------------------------
Best feature set is: employment_numeric + fico 
With: 96.5% accuracy.


Model Comparison:

By adopting the new feature set: employment_numeric + fico, the accuracy increased from 95% to 96.5%, Kappa increased from 0.76 to 0.83, precision increase from 0.78 to 0.82.

We can say that deleting feature "amount", the new two-feature decision tree performed better overall.

**Optimization 3: Decision tree vs. logistic regression classification**

From the second comparision, the best feature set is "numeric employment length + fico score". In the following Decision Tree and Logistic Regression comparison, only the best set will be applied.

Logistic Regression with scikit-learn

In [ ]:
employment_numeric = ["emp"]
fico_features = ["fico"]

feature_sets = {
    "employment_numeric + fico": employment_numeric + fico_features
}

In [ ]:
# Compare feature sets after training each feature set
best = 0
best_name = None
best_actual = None
best_predictions = None

precisions = []
kappas = []
accuracies = []

predictions = {}
actual = None

# For each feature set, we evaluate our model on both the train and the test set
for set_name, feature_set in feature_sets.items():

    # Create a dummyset with only the features in our feature set
    X = lendingclub.loc[:, feature_set]
    X = pd.get_dummies(X)

    y = lendingclub["outcome"]

    # Train model
    classifier = LogisticRegression(penalty="l2", solver="lbfgs", max_iter=10000, random_state=123)
    model = classifier.fit(X, y)

    # Get Predictions
    preds = model.predict(X)
    
    # Compute and print metrics
    accuracy = 100*accuracy_score(y, preds)
    kappa = cohen_kappa_score(y, preds)
    precision = precision_score(y, preds, pos_label= "accept")
    recall = recall_score(y, preds, pos_label= "accept")
    print(f"Accuracy: {accuracy:.1f} Kappa: {kappa:.3f} Precision: {precision:.3f} Recall: {recall:.3f}")  

    # The printed result includes performance of the feature set and the confusion matrix
    print(f"Results for {set_name}:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Accuracy: {accuracy:.1f} Kappa: {kappa:.3f} Precision: {precision:.3f}")
    precisions.append(precision)
    kappas.append(kappa)
    accuracies.append(accuracy)
    print("------------------------")

Accuracy: 88.5 Kappa: 0.252 Precision: 0.491 Recall: 0.221
Results for employment_numeric + fico:
[[ 1868     4]
 [ 1164 13185]]
Accuracy: 88.5 Kappa: 0.252 Precision: 0.491
------------------------


Comparisons Between Decision Tree and Logistic Regression:

The performance of the current model was not improving by switching from Decision Tree to Logistic Regression. The accuracy decline from 95% to 88.5%, Kappa dropped from 0.76 to 0.25, precision dropped from 0.78 to 0.49.

**Ranking**

1. Logistic Regression Model
2. Feature sets
3. Training set size

The following table provided comparision of the three adjustments. We are able to tell that switching to Logistic Regression model has the most impact on the performance of the model, though the effect is negative. Chaning feature set has slightly more significant effect than changing training set size.

![alt text](https://drive.google.com/uc?id=1LTxLyC2fGy0EZAJeV5JXUhOB6ekjuN4W)


# Question 3

Report the best-performing model and the total set of choices that you made in the options above to get to that level of performance. Compare that model performance to the result you reported in Homework 1. 

   - Describe one hypothetical business context for LendingClub where the amount of improvement reported in Question 3 would be worth the extra time and effort needed for optimization.
   - Describe one hypothetical business context where the improvement would **not** have been worth the extra time and effort.

Untill now, the best performing model is the Decision Tree model using "numeric employment length" and "fico" as features set. In the model, the training set contains 80% of total amount of data.
Table 2 contains information of the model comparison.

![alt text](https://drive.google.com/uc?id=1KZQajBiaXAEbhvH7H8k8r1dpcsPub3I9)



   - Since the optimized model performed better overall, the marginal improved accuracy would lead to less loan inefficiency, which is beneficial to LendingClub. Also, the new model requires less information of borrowers, it is more efficient to adopt the new model.
   - However, we noticed that although the second model performed better, the two models are rather similar, with only slight improvement has been made. If the amount of loan is generally small, in other words, the company would not experience huge fund loss caused by the 1% accuracy difference, then it is not worthy to revise the model in changed of slightly higher accuracy.



# Extra Credit

For up to 2 points of extra credit, complete all three of the analyses in Question 1.

For up to 4 points of extra credit, complete all five of the analyses in Question 2.

# Scoring Rubric
![](https://drive.google.com/uc?export=view&id=1dLF4RcciHT_giZEXWio1D0hwK73tKYjK)

![](https://drive.google.com/uc?export=view&id=1nYN7X847FeHtE-0LVpqhQk8iq2YEWx2d)